# Plot ImageCollection

The `geetools` extention contains a set of functions for rendering charts from the results of spatiotemporal reduction of images within an `ee.ImageCollection`. The choice of function dictates the arrangement of data in the chart, i.e., what defines x- and y-axis values and what defines the series. Use the following function descriptions and examples to determine the best function for your purpose.

In [ ]:
import ee, geetools
from geetools.utils import initialize_documentation

initialize_documentation()

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/ipygee/blob/main/docs/usage/plot/plot-imagecollection.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/ipygee/blob/main/docs/usage/plot/plot-imagecollection.ipynb)

## Set up environment

Install all the required libs if necessary and perform the import satements upstream.

In [ ]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [ ]:
from matplotlib import pyplot as plt
from datetime import datetime as dt

import ee
import ipygee #noqa: F401
from bokeh.io import output_notebook

output_notebook()

In [ ]:
# uncomment if authetication to GEE is needed
# ee.Authenticate()
# ee.Initialize(project="<your_project>")

## Example data 

The following examples rely on a `ee.FeatureCollection` composed of three ecoregion features that define regions by which to reduce image data. The ImageCollection data loads the modis vegetation indicies and subset the 2010 2020 decade of images.

In [ ]:
## Import the example feature collection and drop the data property.
ecoregions = (
    ee.FeatureCollection("projects/google/charts_feature_example")
    .select(["label", "value", "warm"])
)


## Load MODIS vegetation indices data and subset a decade of images.
vegIndices = (
    ee.ImageCollection("MODIS/061/MOD13A1")
    .filter(ee.Filter.date("2010-01-01", "2020-01-01"))
    .select(["NDVI", "EVI"])
)

## Plot dates

The `plot_dates*` methods will plot the values of the image collection using their dates as x-axis values.

### series by bands 

Image date is plotted along the x-axis according to the `dateProperty` property. Series are defined by image bands. Y-axis values are the reduction of images, by date, for a single region.

In [ ]:
from bokeh.plotting import figure, show

fig = figure(width=800, height=400)

# Sample data (replace these with your actual data)
dates = ["date1", "date2", "date3"]
ticker_values = list(range(len(dates)))
b1 = [1, 2, 1]
b2 = [2, 3, 2]
b3 = [3, 4, 3]

# Create the plot
fig.line(x=ticker_values, y=b1, legend_label="b1", color="#1d6b99")
fig.line(x=ticker_values, y=b2, legend_label="b2", color="#cf513e")
fig.line(x=ticker_values, y=b3, legend_label="b3", color="#f0af07")

# Add titles and labels
fig.title.text = "Single-region spatial reduction"
fig.xaxis.axis_label = "Image date"
fig.yaxis.axis_label = "Spatial reduction"
fig.y_range.start = 0
fig.y_range.end = 5
fig.legend.title = "Band names"
fig.legend.location = "top_right"
fig.xaxis.ticker = ticker_values
fig.xaxis.major_label_overrides = {i: date for i, date in enumerate(dates)}
fig.xgrid.grid_line_color = None
fig.legend.orientation = "horizontal"
fig.outline_line_color = None

show(fig)

Use `plot_series_by_bands` to display an image time series for a given region; each image band is presented as a unique series. It is useful for comparing the time series of individual image bands. Here, a MODIS image collection with bands representing NDVI and EVI vegetation indices are plotted. The date of every image observation is included along the x-axis, while the mean reduction of pixels intersecting a forest ecoregion defines the y-axis.

In [ ]:
fig = figure(width=800, height=400)

region = ecoregions.filter(ee.Filter.eq("label", "Forest"))
col = vegIndices.bokeh.plot_dates_by_bands(
    region = region.geometry(),
    reducer = "mean",
    scale = 500,
    bands = ["NDVI", "EVI"],
    figure = fig,
    dateProperty = "system:time_start",
)

# once created the figure can be modified as needed using pure bokeh members
col.children[0].yaxis.axis_label = "Vegetation indices (x1e4)"
col.children[0].title.text = "Average Vegetation index Values by date in the Forest ecoregion"

show(col)

### Plot series by region

Image date is plotted along the x-axis according to the `dateProperty` property. Series are defined by regions. Y-axis values are the reduction of images, by date, for a single image band.

In [ ]:
from bokeh.plotting import figure, show

fig = figure(width=800, height=400)

# Sample data (replace these with your actual data)
dates = ["date1", "date2", "date3"]
ticker_values = list(range(len(dates)))
r1 = [1, 2, 1]
r2 = [2, 3, 2]
r3 = [3, 4, 3]

# Create the plot
fig.line(x=ticker_values, y=r1, legend_label="r1", color="#1d6b99")
fig.line(x=ticker_values, y=r2, legend_label="r2", color="#cf513e")
fig.line(x=ticker_values, y=r3, legend_label="r3", color="#f0af07")

# Add titles and labels
fig.title.text = "Single-band spatial reduction"
fig.xaxis.axis_label = "Image date"
fig.yaxis.axis_label = "Spatial reduction"
fig.y_range.start = 0
fig.y_range.end = 5
fig.legend.title = "Regions"
fig.legend.location = "top_right"
fig.xaxis.ticker = ticker_values
fig.xaxis.major_label_overrides = {i: date for i, date in enumerate(dates)}
fig.xgrid.grid_line_color = None
fig.legend.orientation = "horizontal"
fig.outline_line_color = None

show(fig)

Use `plot_dates_by_regions` to display a single image band time series for multiple regions; each region is presented as a unique series. It is useful for comparing the time series of a single band among several regions. Here, a MODIS image collection representing an NDVI time series is plotted for three ecoregions. The date of every image observation is included along the x-axis, while mean reduction of pixels intersecting forest, desert, and grasslands ecoregions define y-axis series.

In [ ]:
fig = figure(width=800, height=400)

region = ecoregions.filter(ee.Filter.eq("label", "Forest"))
col = vegIndices.bokeh.plot_dates_by_regions(
    band = "NDVI",
    regions = ecoregions,
    label = "label",
    reducer = "mean",
    scale = 500,
    figure = fig,
    dateProperty = "system:time_start",
    colors = ['#f0af07', '#0f8755', '#76b349']
)

# once created the axes can be modified as needed using pure matplotlib functions
col.children[0].yaxis.axis_label = "Vegetation indices (x1e4)"
col.children[0].title.text = "Average Vegetation index Values by date in the Forest ecoregion"

show(col)

## PLot DOY

DOY stands for day of year. The `plot_doyseries*` methods will plot the values of the image collection using the day of year as x-axis values.

Note that `.plot_doyseries*` functions take two reducers: one for region reduction (`regionReducer`) and another for intra-annual coincident day-of-year reduction (`yearReducer`). Examples in the following sections use `ee.Reducer.mean()` as the argument for both of these parameters.

### Plot DOY by bands 

Image day-of-year is plotted along the x-axis according to the `dateProperty` property. Series are defined by image bands. Y-axis values are the reduction of image pixels in a given region, grouped by day-of-year.

In [ ]:
from bokeh.plotting import figure, show

fig = figure(width=800, height=400)

# Sample data (replace these with your actual data)
dates = ["doy1", "doy2", "doy3"]
ticker_values = list(range(len(dates)))
b1 = [1, 2, 1]
b2 = [2, 3, 2]
b3 = [3, 4, 3]

# Create the plot
fig.line(x=ticker_values, y=b1, legend_label="b1", color="#1d6b99")
fig.line(x=ticker_values, y=b2, legend_label="b2", color="#cf513e")
fig.line(x=ticker_values, y=b3, legend_label="b3", color="#f0af07")

# Add titles and labels
fig.title.text = "Single-band spatiotemporal reduction"
fig.xaxis.axis_label = "Image date"
fig.yaxis.axis_label = "Reduced values"
fig.y_range.start = 0
fig.y_range.end = 5
fig.legend.title = "Band names"
fig.legend.location = "top_right"
fig.xaxis.ticker = ticker_values
fig.xaxis.major_label_overrides = {i: date for i, date in enumerate(dates)}
fig.xgrid.grid_line_color = None
fig.legend.orientation = "horizontal"
fig.outline_line_color = None

show(fig)

Use `plot_doy_by_bands` to display a day-of-year time series for a given region; each image band is presented as a unique series. It is useful for reducing observations occurring on the same day-of-year, across multiple years, to compare e.g. average annual NDVI and EVI profiles from MODIS, as in this example.

In [ ]:
fig = figure(width=800, height=400)

vegIndices.bokeh.plot_doy_by_bands(
    region = ecoregions.filter(ee.Filter.eq("label", "Grassland")).geometry(),
    spatialReducer = "mean",
    timeReducer = "mean",
    scale = 500,
    bands = ["NDVI", "EVI"],
    figure = fig,
    dateProperty = "system:time_start",
    colors = ['#e37d05', '#1d6b99']
)

# once created the axes can be modified as needed using pure matplotlib functions
fig.yaxis.axis_label = "Vegetation indices (x1e4)"
fig.title.text = "Average Vegetation index Values by doy in the Grassland ecoregion"

show(fig)

### Plot doy by regions 

Image day-of-year is plotted along the x-axis according to the `dateProperty` property. Series are defined by regions. Y-axis values are the reduction of image pixels in a given region, grouped by day-of-year, for a selected image band.


In [ ]:
from bokeh.plotting import figure, show

fig = figure(width=800, height=400)

# Sample data (replace these with your actual data)
dates = ["doy1", "doy2", "doy3"]
ticker_values = list(range(len(dates)))
r1 = [1, 2, 1]
r2 = [2, 3, 2]
r3 = [3, 4, 3]

# Create the plot
fig.line(x=ticker_values, y=r1, legend_label="r1", color="#1d6b99")
fig.line(x=ticker_values, y=r2, legend_label="r2", color="#cf513e")
fig.line(x=ticker_values, y=r3, legend_label="r3", color="#f0af07")

# Add titles and labels
fig.title.text = "Single-region spatiotemporal reduction"
fig.xaxis.axis_label = "Image date"
fig.yaxis.axis_label = "Reduced values"
fig.y_range.start = 0
fig.y_range.end = 5
fig.legend.title = "Region names"
fig.legend.location = "top_right"
fig.xaxis.ticker = ticker_values
fig.xaxis.major_label_overrides = {i: date for i, date in enumerate(dates)}
fig.xgrid.grid_line_color = None
fig.legend.orientation = "horizontal"
fig.outline_line_color = None

show(fig)

Use `plot_doy_by_regions` to display a single image band day-of-year time series for multiple regions, where each distinct region is presented as a unique series. It is useful for comparing annual single-band time series among regions. For instance, in this example, annual MODIS-derived NDVI profiles for forest, desert, and grassland ecoregions are plotted, providing a convenient comparison of NDVI response by region. Note that intra-annual observations occurring on the same day-of-year are reduced by their mean.

In [ ]:
fig = figure(width=800, height=400)

vegIndices.bokeh.plot_doy_by_regions(
    regions = ecoregions,
    label = "label",
    spatialReducer = "mean",
    timeReducer = "mean",
    scale = 500,
    band = "NDVI",
    figure = fig,
    dateProperty = "system:time_start",
    colors = ['#f0af07', '#0f8755', '#76b349']
)

# once created the axes can be modified as needed using pure matplotlib functions
fig.yaxis.axis_label = "NDVI (x1e4)"
fig.title.text = "Average NDVI Values by doy in each ecoregion"

show(fig)

### plot doy by seasons 

In case the observation you want to analyse are only meaningful on a subset of the year a variant of the previous method allows you to plot the data by season. The season is defined by the `seasonStart` and `seasonEnd` parameters, which are 2 numbers between 1 and 366 representing the start and end of the season. To set them, the user can use the {py:method}`ee.Date.getRelative` or {py:class}`time.struct_time` method to get the day of the year. 

```{note} 
The default season is a year (1, 366).
```

In [ ]:
# reduce the regions to grassland
grassland = ecoregions.filter(ee.Filter.eq("label", "Grassland"))

# for plot speed and lisibility only keep 2 years (2010 and 2020) for the example
indices = vegIndices.filter(
        ee.Filter.Or(
            ee.Filter.date("2012-01-01", "2012-12-31"),
            ee.Filter.date("2019-01-01", "2019-12-31"),
        )
)

In [ ]:
fig = figure(width=800, height=400)

indices.bokeh.plot_doy_by_seasons(
    band = "NDVI",
    region = grassland.geometry(),
    seasonStart = ee.Date("2022-04-15").getRelative("day", "year"),
    seasonEnd = ee.Date("2022-09-15").getRelative("day", "year"),
    reducer = "mean",
    scale = 500,
    figure = fig,
    colors = ['#39a8a7', '#9c4f97']
)

# once created the axes can be modified as needed using pure matplotlib functions
fig.yaxis.axis_label = "NDVI (x1e4)"
fig.title.text = "Average NDVI Values during growing season in Grassland"

show(fig)

In [ ]:
fig = figure(width=800, height=400)

indices.bokeh.plot_doy_by_seasons(
    band = "NDVI",
    region = grassland.geometry(),
    reducer = "mean",
    scale = 500,
    figure = fig,
    colors = ['#39a8a7', '#9c4f97']
)

# once created the axes can be modified as needed using pure matplotlib functions
fig.yaxis.axis_label = "NDVI (x1e4)"
fig.title.text = "Average NDVI Values by years"

show(fig)